# Medical Diagnosis Model Finetuning - Project Overview

---

## What This Notebook Covers

1. **The Research Question** - Why this matters
2. **Project Architecture** - How everything fits together
3. **Experimental Design** - What we're comparing
4. **Quick Setup** - Get started fast
5. **Notebook Series** - What each notebook does

---

## 1. The Research Question 

### The Problem

In healthcare AI, we face a fundamental trade-off:

| Approach | Pros | Cons |
|----------|------|------|
| **Large Models** | More knowledge, better general performance | Slow and expensive |
| **Small Models (3B parameters)** | Fast, cheap, can run locally | Less capable out-of-the-box |

### Our Hypothesis

> **Specialization through finetuning can compensate for smaller model size on domain-specific tasks**

In other words: A 3B model trained on medical data might outperform an 8B model with no medical training!

### Why This Matters

**If small finetuned models win:**
- ✅ Faster inference (2-3x speedup)
- ✅ Lower costs (can run on smaller GPUs)
- ✅ Better privacy (can run on-premise)
- ✅ More sustainable (less energy consumption)

**Real-world impact:**
- Rural hospitals with limited compute
- Privacy-sensitive patient data
- Large-scale deployment (thousands of instances)
- Embedded medical devices

## 2. Project Architecture 🏗️

### High-Level Pipeline

```
┌─────────────────────────────────────────────────────────┐
│  1. DATA LOADING                                        │
│     ├─ MedSynth Dataset (10K medical conversations)     │
│     ├─ Split: 70% train, 15% val, 15% test            │
│     └─ ICD-10 diagnosis codes as labels                │
└─────────────────────────────────────────────────────────┘
                        ↓
┌─────────────────────────────────────────────────────────┐
│  2. DATA PROCESSING                                     │
│     ├─ Tokenization (text → numbers)                   │
│     ├─ Chat template formatting                         │
│     └─ Padding & truncation                            │
└─────────────────────────────────────────────────────────┘
                        ↓
        ┌───────────────┴───────────────┐
        ↓                               ↓
┌──────────────────┐          ┌──────────────────┐
│  3a. LLM EVAL    │          │  3b. SLM TRAIN   │
│  (Zero-Shot)     │          │  (LoRA Finetune) │
│                  │          │                  │
│  • Llama 8B      │          │  • Llama 3B      │
│  • Mistral 7B    │          │  • Qwen 3B       │
└──────────────────┘          └──────────────────┘
        ↓                               ↓
┌──────────────────┐          ┌──────────────────┐
│  4a. LLM Results │          │  4b. SLM Eval    │
│  Accuracy, F1... │          │  Accuracy, F1... │
└──────────────────┘          └──────────────────┘
        ↓                               ↓
        └───────────────┬───────────────┘
                        ↓
┌─────────────────────────────────────────────────────────┐
│  5. COMPARISON & ANALYSIS                               │
│     ├─ Size vs Specialization                          │
│     ├─ Performance metrics                              │
│     ├─ Visualizations                                   │
│     └─ Conclusions                                      │
└─────────────────────────────────────────────────────────┘
```

## 3. Experimental Design

### Models Being Compared

#### LLMs (Large, Untrained)

**Purpose:** Baseline performance with size advantage

| Model | Size | Training | Use Case |
|-------|------|----------|----------|
| Meta-Llama-3.1-8B | 8B | Zero-shot | Large reference |
| Mistral-7B | 7B | Zero-shot | Medium reference |

**Strengths:**
- More parameters = more knowledge
- No training needed
- General-purpose capability

**Weaknesses:**
- Slow (8-10 tokens/sec)
- Memory-intensive
- No domain specialization

#### SLMs (Small, Finetuned)

**Purpose:** Test if specialization beats size

| Model | Size | Training | Use Case |
|-------|------|----------|----------|
| Llama-3.2-3B | 3B | LoRA finetuned | Specialized model #1 |
| Qwen2.5-3B | 3B | LoRA finetuned | Specialized model #2 |

**Strengths:**
- Fast (20-30 tokens/sec)
- Memory-efficient
- Domain-specialized

**Weaknesses:**
- Requires training
- Fewer parameters
- Limited to medical domain

### Task: ICD-10 Code Classification

**Input:** Doctor-patient conversation

```
Doctor: What brings you here today?
Patient: I have a sore throat and runny nose for 3 days.
Doctor: Do you have fever?
Patient: Yes, 38.5°C yesterday.
...
```

**Output:** ICD-10 diagnosis code

```
J06.9  (Acute upper respiratory infection, unspecified)
```

**Why ICD-10?**
- International standard for diagnoses
- Used worldwide in healthcare systems
- Clear ground truth for evaluation
- Hierarchical structure (J06.9 → J06 → J)

### Training Method: LoRA (Low-Rank Adaptation)

**Why LoRA?**

Traditional finetuning trains ALL parameters:
```
3B model = 3,000,000,000 trainable parameters
Memory needed: ~12GB VRAM
Training time: ~10 hours
```

LoRA trains only small adapter matrices:
```
LoRA adapters = ~10,000,000 trainable parameters (0.3%!)
Memory needed: ~6GB VRAM
Training time: ~2 hours
```

**How LoRA Works:**

```
Original weight matrix W:  [4096 x 4096] = 16M parameters ❌

LoRA decomposition:
  A matrix: [4096 x 64]  = 262K parameters ✅
  B matrix: [64 x 4096]  = 262K parameters ✅
  Total:                 = 524K parameters (97% reduction!)

Updated weight: W' = W + A × B
```

**Benefits:**
- ✅ 97% fewer trainable parameters
- ✅ 5x faster training
- ✅ 50% less VRAM needed
- ✅ Can train on consumer GPUs

## 4. Quick Setup

### Prerequisites

```bash
# GPU with CUDA (recommended: 16GB+ VRAM)
nvidia-smi

# Python 3.11+
python --version

# Install dependencies
pip install -e ".[dev]"
```

### Environment Setup

**IMPORTANT:** Set this before running anything!

In [1]:
import os

# Critical for memory management on modern GPUs
os.environ['PYTORCH_ALLOC_CONF'] = 'expandable_segments:True'

print("Environment configured for optimal GPU memory usage")

Environment configured for optimal GPU memory usage


### Check GPU

In [2]:
import torch

if torch.cuda.is_available():
    print(f"CUDA Available: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("❌ CUDA not available - will use CPU (very slow!)")

❌ CUDA not available - will use CPU (very slow!)


### Project Structure

In [3]:
from pathlib import Path

# Add src to path for imports
import sys
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

print(f"Project Root: {project_root}")
print(f"\nKey Directories:")
for dir_name in ['src', 'data', 'models', 'outputs', 'notebooks']:
    dir_path = project_root / dir_name
    exists = "✅" if dir_path.exists() else "❌"
    print(f"  {exists} {dir_name}/")

Project Root: /Users/qtf7046/uni/finetuning

Key Directories:
  ✅ src/
  ✅ data/
  ✅ models/
  ✅ outputs/
  ✅ notebooks/


## 5. Notebook Series

### Learning Path

Follow these notebooks in order to understand the complete pipeline:

#### **01_Data_Loading_and_Exploration.ipynb**
**What you'll learn:**
- How to load medical conversation data
- Dataset statistics and distribution
- ICD-10 code analysis
- Data quality checks

**Key concepts:**
- HuggingFace datasets
- Train/val/test splitting
- Data exploration with pandas

---

#### **02_Data_Processing_and_Tokenization.ipynb**
**What you'll learn:**
- How text becomes numbers (tokenization)
- Chat templates and formatting
- Padding and truncation strategies
- Processing pipeline

**Key concepts:**
- Tokenizers
- Special tokens
- Sequence length management

---

#### **03_LLM_Evaluation_ZeroShot.ipynb**
**What you'll learn:**
- Loading large language models
- Zero-shot inference
- Evaluation metrics (accuracy, F1, etc.)
- Performance analysis

**Key concepts:**
- Model quantization (4-bit)
- Batch inference
- Metrics computation

---

#### **04_SLM_Training_LoRA.ipynb**
**What you'll learn:**
- What is LoRA and why it's efficient
- Setting up training
- Training loop explained
- Monitoring progress
- Saving checkpoints

**Key concepts:**
- Parameter-efficient finetuning
- Learning rate scheduling
- Early stopping

---

#### **05_SLM_Evaluation_Finetuned.ipynb**
**What you'll learn:**
- Loading finetuned models
- Evaluating performance
- Comparing before/after finetuning
- Understanding improvements

**Key concepts:**
- Model loading from checkpoints
- Adapter merging
- Performance metrics

---

#### **06_Results_Analysis_and_Comparison.ipynb**
**What you'll learn:**
- Statistical analysis of results
- Size vs Specialization comparison
- Creating publication-quality visualizations
- Drawing conclusions

**Key concepts:**
- Statistical tests
- Data visualization
- Scientific reporting

---

#### **07_Custom_Interview_Testing.ipynb**
**What you'll learn:**
- Interactive model testing
- Creating custom test cases
- Comparing model predictions
- Error analysis

**Key concepts:**
- Interactive widgets
- Real-world testing
- Model comparison

## 6. Expected Results

### Performance Metrics We Track

**Accuracy Metrics:**
- Exact Match Accuracy (full ICD code correct)
- Prefix Match (category correct)
- Top-K Accuracy (correct code in top predictions)

**Statistical Metrics:**
- Precision (how many predictions are correct)
- Recall (how many actual cases found)
- F1 Score (harmonic mean of precision/recall)

**Performance Metrics:**
- Inference latency (time per prediction)
- Throughput (predictions per second)
- Memory usage (VRAM required)

### Possible Outcomes

**Scenario 1: SLMs Win 🎯**
```
Llama 3B (finetuned):  Accuracy: 85%
Llama 8B (untrained):  Accuracy: 75%
```
**Conclusion:** Specialization > Size for medical tasks
**Recommendation:** Deploy small finetuned models

**Scenario 2: LLMs Win 📈**
```
Llama 8B (untrained):  Accuracy: 85%
Llama 3B (finetuned):  Accuracy: 75%
```
**Conclusion:** Size > Specialization, need more parameters
**Recommendation:** Use larger models or improve finetuning

**Scenario 3: Mixed Results 🤔**
```
Common diagnoses:  SLMs better (learned from training data)
Rare diagnoses:    LLMs better (general knowledge helps)
```
**Conclusion:** Different models for different use cases
**Recommendation:** Hybrid system or ensemble

## 8. Key Takeaways

### What Makes This Project Interesting?

1. **Practical Research Question**
   - Real-world deployment considerations
   - Trade-offs everyone faces
   - Actionable conclusions

2. **Clean Experimental Design**
   - Clear comparison groups
   - Controlled variables
   - Reproducible methodology

3. **State-of-the-Art Techniques**
   - Parameter-efficient finetuning (LoRA)
   - Modern LLMs (Llama 3, Qwen 2.5)
   - Best practices (caching, monitoring, etc.)

4. **Production-Ready Code**
   - Modular architecture
   - Proper error handling
   - Memory management
   - Comprehensive logging

### Skills You'll Learn

**Machine Learning:**
- Language model finetuning
- Transfer learning
- Evaluation methodologies
- Hyperparameter tuning

**Engineering:**
- GPU optimization
- Memory management
- Batch processing
- Caching strategies

**Research:**
- Experimental design
- Statistical analysis
- Scientific visualization
- Result interpretation

## 9. Next Steps

### Ready to Start?

1. **Make sure environment is set up:**
   ```bash
   python -c "import torch; print('CUDA:', torch.cuda.is_available())"
   ```

2. **Run the setup cell above** to configure paths

3. **Open notebook 01:**
   ```
   01_Data_Loading_and_Exploration.ipynb
   ```

4. **Follow along step-by-step!**

### Questions?

- **Documentation:** See `GUIDE.md` for full project documentation
- **Quick Start:** See `QUICKSTART.md` for fast setup
- **Architecture:** See `ARCHITECTURE_REDESIGN.md` for design decisions

---

**Let's dive in!** 🚀